In [ ]:
import music21 as mu
import pandas as pd
import numpy as np
from data_helper import *

def count_match(notes, fingers):
    """
    PIGデータと運指配列との一致数を返す
    """
    match = 0
    for i in range(len(notes)):
        if notes.iloc[i].finger == fingers.iat[i]:
            match += 1
    return match

def noteWithFingering(note, pig, finger):
    note.articulations.append(mu.articulations.Fingering(finger))
    if pig.finger != finger:
        note.style.color = "red"
        note.addLyric(pig._finger)

def noteColoringByFingerMatches(note, fingers):
    differ = (len(set(fingers)) - 1) / (len(fingers) - 1)
    color = int((1 - pow(1 - differ, 2)) * 255)
    red = "{:02x}".format(color)
    note.style.color = f"#{red}0000"
    
def drawFingerings(stream, pig_notes, fingers):
    appendPIGNotes(stream, pig_notes, lambda i, n, c: noteWithFingering(n, c, fingers.iat[i]))

def drawMatches(stream, pig_notes, fingerZipped):
    fingerlist = list(fingerZipped)
    appendPIGNotes(stream, pig_notes, lambda i, n, c: noteColoringByFingerMatches(n, fingerlist[i]))

In [ ]:
convert_fingering_to_PIG(
    './data/001-my_fingering.csv',
    './data/001-my_fingering_R.txt', 
    './data/001-my_fingering_L.txt', 
    "./tmp/PianoFingeringDataset/FingeringFiles/001-1_fingering.txt")


In [ ]:
table = read_PIG("./tmp/PianoFingeringDataset/FingeringFiles/001-1_fingering.txt")
table2 = read_PIG("./tmp/PianoFingeringDataset/FingeringFiles/001-2_fingering.txt")
table5 = read_PIG("./tmp/PianoFingeringDataset/FingeringFiles/001-5_fingering.txt")
table8 = read_PIG("./tmp/PianoFingeringDataset/FingeringFiles/001-8_fingering.txt")

czernyR = read_fingering('./data/001-Czerny_fingering_R.txt')
czernyL = read_fingering('./data/001-Czerny_fingering_L.txt')

In [ ]:
rights = table[table.ch == 0]
pr = mu.stream.Part()
drawFingerings(pr, rights, czernyR)

lefts = table[table.ch == 1]
pl = mu.stream.Part()
drawFingerings(pl, lefts, czernyL)

score = mu.stream.Score()
score.insert(0, pr)
score.insert(0, pl)

score.write('musicxml', './mxl/bach_invention1.xml')
#m = count_match(rights, czernyR) + count_match(lefts, czernyL)
#print(f"{m} / {len(rights) + len(lefts)}")

In [ ]:
#fingersr = zip(table[table.ch == 0].finger, table2[table.ch == 0].finger, table5[table.ch == 0].finger, table8[table.ch == 0].finger)
#fingersl = zip(table[table.ch == 1].finger, table2[table.ch == 1].finger, table5[table.ch == 1].finger, table8[table.ch == 1].finger)

rights = table[table.ch == 0]
pr = mu.stream.Part()
drawMatches(pr, rights, zip(table[table.ch == 0].finger, czernyR))

score = mu.stream.Score()
score.insert(0, pr)
score.write('musicxml', './mxl/PIG_differs2.xml')

#m = match(rights, czernyR) + match(lefts, czernyL)
#print(f"{m} / {len(rights) + len(lefts)}")

In [ ]:
def make_match_matrix(pig_array):
    df = pd.DataFrame(columns = range(len(pig_array)))
    pig_fingers = [table.finger for table in pig_array]
    data_length = len(pig_fingers[0])
    for i in range(len(pig_fingers)):
        row = {}
        for j in range(len(pig_fingers)):
            fingersr = zip(pig_fingers[i], pig_fingers[j]) # make pair
            cnt = 0
            for _, fs in enumerate(fingersr):
                if len(set(fs)) < len(fs):
                    cnt += 1
            row[j] = cnt / data_length
            #rate = "{:.3f}".format(cnt / data_length)
        df = df.append(row, ignore_index = True)
    print(df)

pig = [table, table2, table5, table8]
make_match_matrix(pig)